# YOLO Image Annotation (hbmon workflow)

This notebook mirrors the hbmon production detection flow as closely as possible for a **single image**.
It runs multiple YOLO model sizes against the same input, annotates detections, and optionally saves
annotated outputs for offline review.


In [ ]:
from pathlib import Path
import resource
import time

import cv2
from IPython.display import display
from PIL import Image
from ultralytics import YOLO

from hbmon.config import Settings
from hbmon.worker import (
    DEFAULT_BIRD_CLASS_ID,
    _collect_bird_detections,
    _draw_bbox,
    _draw_text_lines,
    _write_jpeg,
)
from hbmon.yolo_utils import resolve_predict_imgsz


def load_image_bgr(image_path: Path):
    if not image_path.exists():
        raise FileNotFoundError(f"Image not found: {image_path}")
    frame = cv2.imread(str(image_path))
    if frame is None:
        raise RuntimeError(f"OpenCV failed to load: {image_path}")
    return image_path, frame


def annotate_detections(frame_bgr, detections, label_lines=None):
    annotated = frame_bgr.copy()
    for det in detections:
        annotated = _draw_bbox(annotated, det)
    if label_lines:
        annotated = _draw_text_lines(annotated, label_lines)
    return annotated


def _format_rss_kb(rss_kb: int) -> str:
    return f"{rss_kb / 1024:.1f} MB"


def collect_resource_snapshot() -> dict[str, float | int]:
    usage = resource.getrusage(resource.RUSAGE_SELF)
    rss_kb = int(usage.ru_maxrss)
    return {
        "cpu_time_s": usage.ru_utime + usage.ru_stime,
        "max_rss_kb": rss_kb,
    }


def run_yolo_model(
    model_name: str,
    frame_bgr,
    settings: Settings,
    imgsz_env: str = "auto",
    bird_class_id: int = DEFAULT_BIRD_CLASS_ID,
):
    yolo = YOLO(model_name, task="detect")
    imgsz = resolve_predict_imgsz(imgsz_env, frame_bgr.shape[:2])
    results = yolo.predict(
        frame_bgr,
        conf=float(settings.detect_conf),
        iou=float(settings.detect_iou),
        classes=[bird_class_id],
        imgsz=imgsz,
        verbose=False,
    )
    detections = _collect_bird_detections(
        results,
        int(settings.min_box_area),
        bird_class_id,
    )
    label_lines = [
        f"Model: {model_name}",
        f"Detections: {len(detections)}",
        f"Conf/IoU: {settings.detect_conf:.2f}/{settings.detect_iou:.2f}",
        f"Min box area: {settings.min_box_area}px",
    ]
    annotated = annotate_detections(frame_bgr, detections, label_lines=label_lines)
    return {
        "model": model_name,
        "detections": detections,
        "annotated": annotated,
    }


In [ ]:
# Update this path to point at your local image.
image_path = Path("/path/to/your/image.jpg")

# YOLO model sizes to compare.
yolo_models = ["yolo11n.pt", "yolo11s.pt", "yolo11m.pt", "yolo11l.pt"]

# Use the same defaults as hbmon (with env overrides if set).
settings = Settings().with_env_overrides()

# Match hbmon's default imgsz behavior.
imgsz_env = "auto"

# Save annotated outputs (optional).
save_outputs = False
output_dir = Path("annotated_outputs")


In [ ]:
image_path, frame_bgr = load_image_bgr(image_path)

results = []
for model_name in yolo_models:
    print(f"Running {model_name}...")

    start_time = time.perf_counter()
    start_snapshot = collect_resource_snapshot()

    result = run_yolo_model(model_name, frame_bgr, settings, imgsz_env=imgsz_env)
    results.append(result)

    end_time = time.perf_counter()
    end_snapshot = collect_resource_snapshot()

    elapsed_s = end_time - start_time
    cpu_delta_s = end_snapshot["cpu_time_s"] - start_snapshot["cpu_time_s"]
    max_rss_delta_kb = end_snapshot["max_rss_kb"] - start_snapshot["max_rss_kb"]

    print(
        "Timing: {:.3f}s wall, {:.3f}s CPU | Max RSS: {} (Δ {})".format(
            elapsed_s,
            cpu_delta_s,
            _format_rss_kb(end_snapshot["max_rss_kb"]),
            _format_rss_kb(max_rss_delta_kb),
        )
    )

    annotated_rgb = cv2.cvtColor(result["annotated"], cv2.COLOR_BGR2RGB)
    display(Image.fromarray(annotated_rgb))

    if save_outputs:
        output_dir.mkdir(parents=True, exist_ok=True)
        output_path = output_dir / f"{image_path.stem}_{model_name.replace('.pt', '')}.jpg"
        _write_jpeg(output_path, result["annotated"])
        print(f"Saved: {output_path}")
